In [1]:
!pip install eli5

  Using cached https://files.pythonhosted.org/packages/97/2f/c85c7d8f8548e460829971785347e14e45fa5c6617da374711dec8cb38cc/eli5-0.10.1-py2.py3-none-any.whl


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [5]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [7]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [9]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [10]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

In [11]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [12]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [13]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [14]:
df.features[0]

'[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [0]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [16]:
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [17]:
literal_eval(str_dict)[0]['key']

'Gender'

In [18]:
literal_eval(str_dict)[0]['value']

['Men']

In [19]:
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  if str(x) == 'nan': return []
  return literal_eval(x.replace('\\"','"'))

df['features_parsed'] = df['features'].map(parse_features)

In [21]:
df['features_parsed'].head()

0    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
1    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
2    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
3    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
4    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
Name: features_parsed, dtype: object

In [22]:
df['features_parsed'].head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [24]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [25]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

len(keys)

476

In [26]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map( lambda feats: feats[key] if key in feats else np.nan)

In [27]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_fits most screen size', 'feat_����������������',
       'feat_atpv arc rating', 'feat_animal type', 'feat_item style',
       'feat_battery voltage', 'feat_display', 'feat_what includes',
       'feat_is polarized', 'feat_country of manufacturer'],
      dtype='object', length=526)

In [28]:
df[ df['feat_athlete'].isnull() ].shape[0]

18272

In [29]:
df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [31]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df[ 'feat_brand' ].factorize()[0]
df['feat_color_cat'] = df[ 'feat_color' ].factorize()[0]
df['feat_gender_cat'] = df[ 'feat_gender' ].factorize()[0]
df['feat_manufacturer part number_cat'] = df[ 'feat_manufacturer part number' ].factorize()[0]
df['feat_material_cat'] = df[ 'feat_material' ].factorize()[0]

df['feat_sport_cat'] = df[ 'feat_sport' ].factorize()[0]
df['feat_style_cat'] = df[ 'feat_style' ].factorize()[0]

for key in keys:
    df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [33]:
df[ df.brand != df.feat_brand ].shape

(18228, 531)

In [34]:
df[ df.brand == df.feat_brand ].shape

(52, 531)

In [35]:
df[ df.brand == df.feat_brand ].head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,...,feat_metal stamp,feat_main colour,feat_ground,feat_toe type,feat_athlete,feat_mpn,feat_profession,feat_display technology,feat_case thickness,feat_front style,feat_name,feat_is dark sky-compliant,feat_packageweight,feat_kids backpacks,feat_has mercury,feat_case material,feat_retail price,feat_band type,feat_case type:,feat_length,feat_product id,feat_article,feat_features & fastening,feat_country/region of manufacture,feat_label,feat_fits most screen size,feat_����������������,feat_atpv arc rating,feat_animal type,feat_item style,feat_battery voltage,feat_display,feat_what includes,feat_is polarized,feat_country of manufacturer,feat_brand_cat,feat_color_cat,feat_gender_cat,feat_manufacturer part number_cat,feat_material_cat
2163,AVpfCJDxLJeJML43067z,NaN,adidas,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-10-05T16:56:22Z,2016-10-05T16:56:22Z,"[{""dateSeen"":[""2016-10-05T16:56:22Z""],""sourceU...",NaN,NaN,"[{""key"":""Assembled Product Dimensions (L x W x...",NaN,https://i5.walmartimages.com/asr/4d0f0c90-a3db...,NaN,886398578150,Adidas,NaN,"[{""dateSeen"":[""2016-10-05T16:56:22Z""],""name"":""...",Adidas Performance Men's Superstar 3g Slide Sa...,39.99,39.99,NaN,NaN,new,NaN,USD,2016-10-05T16:56:22Z,2016-10-04T00:00:00Z,NaN,False,ULTIMATE SHOE ZONE - Walmart.com,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/adidas-Performance-...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244,1,0,-1,-1
2417,AVpe-Esx1cnluZ0-bAGl,NaN,adidas,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-06-08T05:30:57Z,2016-06-08T05:30:57Z,"[{""dateSeen"":[""2016-06-08T05:30:57Z""],""sourceU...",NaN,8.873834e+11,"[{""key"":""Shoe Size"",""value"":[""11.5""]},{""key"":""...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-32f5/...,NaN,"887383430996,0887383430996",NaN,NaN,"[{""dateSeen"":[""2016-06-08T05:30:57Z""],""name"":""...",Adidas Originals Hardcourt Block Men's Hightop...,59.99,59.99,NaN,NaN,new,NaN,USD,2016-06-08T05:30:57Z,2016-06-07T00:00:00Z,NaN,False,Street Moda - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Adidas-Originals-Har...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244,1,0,-1,-1
3021,AVpfACB3ilAPnD_xSu5G,NaN,adidas,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-06-08T05:31:25Z,2016-06-08T05:31:25Z,"[{""dateSeen"":[""2016-06-08T05:31:25Z""],""sourceU...",NaN,8.877800e+11,"[{""key"":""Shoe Size"",""value"":[""13""]},{""key"":""Ge...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-18ec/...,NaN,"0887780011002,887780011002",NaN,NaN,"[{""dateSeen"":[""2016-06-08T05:31:25Z""],""name"":""...",Adidas Originals Men's Stan Smith Tennis Sneak...,69.99,69.99,NaN,NaN,new,NaN,USD,2016-06-08T05:31:25Z,2016-06-07T00:00:00Z,NaN,False,Street Moda - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Adidas-Originals-Men...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244,77,0,-1,-1
3054,AVpe_gllilAPnD_xSjE6,NaN,adidas,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-08T05:31:12Z,2016-07-09T01:27:09Z,"[{""dateSeen"":[""2016-07-09T01:27:09Z""],""sourceU...",NaN,8.873837e+11,"[{""key"":""Shoe Size"",""value"":[""11.5""]},{""key"":""...",NaN,http://i5.walmartimages.com/asr/0d03c9ca-15fe-...,NaN,"887383735985,088

In [36]:
df[ df.brand == df.feat_brand ][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [37]:
df[ df.brand == df.feat_brand ][ ['brand', 'feat_brand'] ].values

array([['adidas', 'adidas'],
       ['adidas', 'adidas'],
       ['adidas', 'adidas'],
       ['adidas', 'adidas'],
       ['totes', 'totes'],
       ['totes', 'totes'],
       ['totes', 'totes'],
       ['totes', 'totes'],
       ['totes', 'totes'],
       ['totes', 'totes'],
       ['totes', 'totes'],
       ['totes', 'totes'],
       ['1031', '1031'],
       ['elope', 'elope'],
       ['elope', 'elope'],
       ['elope', 'elope'],
       ['elope', 'elope'],
       ['elope', 'elope'],
       ['elope', 'elope'],
       ['elope', 'elope'],
       ['elope', 'elope'],
       ['adidas', 'adidas'],
       ['adidas', 'adidas'],
       ['adidas', 'adidas'],
       ['adidas', 'adidas'],
       ['20-001707000', '20-001707000'],
       ['20-001707000', '20-001707000'],
       ['20-001707000', '20-001707000'],
       ['adidas', 'adidas'],
       ['funworld', 'funworld'],
       ['funworld', 'funworld'],
       ['funworld', 'funworld'],
       ['adidas', 'adidas'],
       ['adidas', 'adidas'],
  

In [38]:
df[ df.brand != df.feat_brand ][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [0]:
df['brand'] = df['brand'].map( lambda x: str(x).lower() )

In [40]:
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = ['']

In [42]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.406681914979096, 4.201368327561316)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat']

In [44]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.095036489437256, 4.257969685988902)

In [45]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat


In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat']

In [47]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2581 ± 0.0103,brand_cat
0.1015 ± 0.0115,feat_material_cat
0.0438 ± 0.0020,feat_gender_cat
0.0204 ± 0.0003,feat_brand_cat
0.0065 ± 0.0013,feat_color_cat


In [48]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.05217729279499, 4.172929380291131)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']

In [50]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.21805004511301, 4.263219984021539)

In [51]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2595 ± 0.0080,brand_cat
0.1099 ± 0.0134,feat_material_cat
0.0435 ± 0.0055,feat_gender_cat
0.0224 ± 0.0007,feat_brand_cat


In [52]:
df['brand'].value_counts()

nike                1777
puma                 609
ralph lauren         526
vans                 386
new balance          371
                    ... 
gargoyles              1
spiewak                1
coool                  1
salomon, salomon       1
fit & fresh            1
Name: brand, Length: 1732, dtype: int64

In [53]:
df['brand'].value_counts(normalize=True)

nike                0.097210
puma                0.033315
ralph lauren        0.028775
vans                0.021116
new balance         0.020295
                      ...   
gargoyles           0.000055
spiewak             0.000055
coool               0.000055
salomon, salomon    0.000055
fit & fresh         0.000055
Name: brand, Length: 1732, dtype: float64

In [54]:
df[ df['brand'] == 'nike' ].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat']

In [57]:
run_model(feats, model)

(-57.221957750388256, 4.178480089180036)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat']

In [59]:
run_model(feats, model)

(-57.08194142961626, 4.247173045108944)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']

In [61]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2600 ± 0.0110,brand_cat
0.1047 ± 0.0084,feat_material_cat
0.0458 ± 0.0047,feat_gender_cat
0.0213 ± 0.0011,feat_brand_cat
0.0069 ± 0.0016,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [62]:
run_model(feats, model)

(-57.090032746695655, 4.295805328656265)

In [65]:
df[ df['brand'] == 'nike' ].features_parsed.sample(5).values

array([{'style': 'shirts & tops', 'sleeve length': 'short sleeve', 'quantity in set': '1', 'condition': 'new with tags', 'item type': 'apparel'},
       {'style': 'athletic sneakers', 'condition': 'new with box'},
       {'manufacturer part number': 'nikm-paulrodriguez-cybergre320.9m', 'gender': 'men', 'shoe category': "men's shoes", 'features': 'durable sole ,�� comfortable fit', 'brand': 'nike', 'model': 'nikm-paulrodriguez-cybergre320.9m', 'casual & dress shoe style': 'skateboarding shoes'},
       {'sport': 'soccer', 'shade': 'white-hyper turquoise', 'condition': 'new with box'},
       {'sport': 'training shoes', 'occasion': 'training shoes', 'material': 'suede', 'gender': 'men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'suede', 'color': 'anthracite/lyon blue/blk/white', 'model': '644843 001', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'cross trainer shoes', 'manufacturer part number': 

In [0]:
feats_cat = [x for x in df.columns if 'cat in x']

In [68]:
feats_cat

['id',
 'asins',
 'brand',
 'categories',
 'colors',
 'count',
 'dateadded',
 'dateupdated',
 'descriptions',
 'dimension',
 'ean',
 'features',
 'flavors',
 'imageurls',
 'isbn',
 'keys',
 'manufacturer',
 'manufacturernumber',
 'merchants',
 'name',
 'prices_amountmin',
 'prices_amountmax',
 'prices_availability',
 'prices_color',
 'prices_condition',
 'prices_count',
 'prices_currency',
 'prices_dateadded',
 'prices_dateseen',
 'prices_flavor',
 'prices_issale',
 'prices_merchant',
 'prices_offer',
 'prices_returnpolicy',
 'prices_shipping',
 'prices_size',
 'prices_source',
 'prices_sourceurls',
 'prices_warranty',
 'quantities',
 'reviews',
 'sizes',
 'skus',
 'sourceurls',
 'upc',
 'vin',
 'websiteids',
 'weight',
 'brand_cat',
 'features_parsed',
 'feat_sole',
 'feat_is recyclable',
 'feat_autographed',
 'feat_clasp type',
 'feat_charge time (minutes)',
 'feat_date',
 'feat_hunting',
 'feat_product in inches (l x w x h)',
 'feat_type',
 'feat_dial color',
 'feat_gender',
 'feat_

In [0]:
feats += feats_cat

In [0]:
feats = list(set(feats))

In [71]:
model

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [72]:
run_model(feats,model)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'servus'

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'josmo'

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'josmo'

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score

(nan, nan)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat', 'feat_shape_cat', 'feat_metal type_cat']

In [0]:
result = run_model(feats, model)

In [76]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-57.21027409919043, 4.1784449940216)


Weight,Feature
0.2563 ± 0.0078,brand_cat
0.1038 ± 0.0163,feat_material_cat
0.0244 ± 0.0038,feat_gender_cat
0.0179 ± 0.0010,feat_brand_cat
0.0126 ± 0.0004,feat_shape_cat
0.0084 ± 0.0014,feat_metal type_cat
0.0031 ± 0.0014,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [81]:
!git pull

Already up to date.


In [0]:
!git add matrix_one/day5.ipynb

In [83]:
!git commit -m "More advanced ML model on Men's Shoe Prices dataset from data.world."

[master 5c8bf93] More advanced ML model on Men's Shoe Prices dataset from data.world.
 1 file changed, 1 insertion(+)
 create mode 100644 matrix_one/day5.ipynb


In [84]:
!git push -u origin master

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 20.81 KiB | 2.60 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/mpasterz/dw_matrix.git
   7c1da6f..5c8bf93  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.
